In [1]:
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm
import scipy.stats as stats
from matplotlib import gridspec
import aplpy as apl #The Astronomy Plotting Library for python
import astropy
from astropy.stats import sigma_clip
from astropy.modeling import functional_models, models, fitting
import astropy.units as U
from astropy.coordinates import ICRS, Galactic, FK4, FK5, Angle, Latitude, Longitude
import astropy.constants as C
from astropy import wcs
import astropy.io.fits as fits
from astropy.io import ascii
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.stats import signal_to_noise_oir_ccd as SNRas
#import spectral_cube as SC
#from spectral_cube import SpectralCube as sc
from astropy.wcs import WCS
matplotlib.use('Agg')
import matplotlib.cm as cm
#import astrometry as ast
import pyfits
import requests
import json
import os
from sklearn import datasets, linear_model
from scipy.optimize import curve_fit
import numpy.ma as ma
from astropy.table import Table, Column
import pyextract
import sewpy
import pandas as pd
from astropy import units as u
#import alipy

/home/lynge/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
def dist(mid,end):
    return np.sqrt((mid[0] - end[0])**2 + (mid[1] - end[1])**2)

In [3]:
def flux_cal(A_mag,B_mag,B_flux):
    '''Flux of A found through magnitude and flux of B'''
    return B_flux*10**((B_mag-A_mag)/2.5)

In [4]:
def Basic(quasar,header):
    '''Provides a basic list of neccesities from the fits header file for use in the later \
    flux determination, it does not serve an individual purpose beyound that'''
    AGNdata = quasar #np.rot90(np.rot90(np.fliplr(quasar)))
    #header = data[0].header
    #print header
    RAstd = header['CRVAL1']
    DECstd = header['CRVAL2']
    #print RAstd, DECstd
    pixspa = header['PIXSCALE']/(3600.)
    #print pixspa
    RAstdpix = header['CRPIX1']
    DECstdpix = header['CRPIX2']
    exptime = 10 # header['EXPTIME']
    return header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime

In [5]:
def curve(quasar,header,w,center,apparature):
    '''Determines the FLUX of the stellar object given the the numpy array of the image, \
    the header of the fits file, the astropy coordinate representation, \
    the object position and the apparature of interest'''
    header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime = Basic(quasar,header)
    #AGNdata = np.swapaxes(AGNdata,0,1)
    y, x = np.ogrid[0:header['NAXIS1'],0:header['NAXIS1']]
    #w = np.swapaxes(w,0,1)
    #F_AGN = np.zeros((512,512,3)) #((int(2*apparature/pixspa)+1,int(2*apparature/pixspa)+1,3))
    #print center
    x1 = w.all_world2pix(center[0],center[1],0)
    x1,y1 = (x1[0]),(x1[1])
    if 15 < x1 < header['NAXIS1'] - 15 and 15 < y1 < header['NAXIS2'] - 15:
        try:
            mask = ((y-y1)**2 + (x-x1)**2) > (apparature/float(pixspa))**2
            AGNdata[mask] = float(0)
            mask = AGNdata == 0.
            rows = np.flatnonzero((~mask).sum(axis=1))
            cols = np.flatnonzero((~mask).sum(axis=0))
            AGNdata = AGNdata[rows.min():rows.max()+1, cols.min():cols.max()+1]
            #print apparature/float(pixspa)
        except:
            AGNdata = np.array([[float(0),float(0)],[float(0),float(0)]])
    else:
        AGNdata = np.array([[float(0),float(0)],[float(0),float(0)]])
    #if np.sum(AGNdata) < 500:
    #    AGNdata = np.array([[float(0),float(0)],[float(0),float(0)]])
    return AGNdata, x1, y1

In [12]:
quasJ = [os.path.join('/media/lynge/Lynge-Back-up/NGC3783/z',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/NGC3783/z')]

sky_RA_WORLD = SkyCoord('11:38:59 -37:44:44', unit=(u.hourangle, u.deg)).ra.degree #NGC3783
sky_DEC_WORLD = SkyCoord('11:38:59 -37:44:44', unit=(u.hourangle, u.deg)).dec.degree

#sky_RA_WORLD = SkyCoord('22:09:11 -47:09:05', unit=(u.hourangle, u.deg)).ra.degree #NGC7213
#sky_DEC_WORLD = SkyCoord('22:09:11 -47:09:05', unit=(u.hourangle, u.deg)).dec.degree

#sky_RA_WORLD = SkyCoord('00:24:27 07:49:19', unit=(u.hourangle, u.deg)).ra.degree #P525-E
#sky_DEC_WORLD = SkyCoord('00:24:27 07:49:19', unit=(u.hourangle, u.deg)).dec.degree 

#sky_RA_WORLD = SkyCoord('18:45:10 -62:22:42', unit=(u.hourangle, u.deg)).ra.degree #F51
#sky_DEC_WORLD = SkyCoord('18:45:10 -62:22:42', unit=(u.hourangle, u.deg)).dec.degree

#sky_RA_WORLD = SkyCoord('00:55:07 00:42:41', unit=(u.hourangle, u.deg)).ra.degree #SA092_342
#sky_DEC_WORLD = SkyCoord('00:55:07 00:42:41', unit=(u.hourangle, u.deg)).dec.degree

#sky_RA_WORLD = SkyCoord('05:58:05.5 -38:20:43.5', unit=(u.hourangle, u.deg)).ra.degree #H0557
#sky_DEC_WORLD = SkyCoord('05:58:05.5 -38:20:43.5', unit=(u.hourangle, u.deg)).dec.degree

#sky_RA_WORLD = SkyCoord('01:23:47 -58:49:35', unit=(u.hourangle, u.deg)).ra.degree #F9
#sky_DEC_WORLD = SkyCoord('01:23:47 -58:49:35', unit=(u.hourangle, u.deg)).dec.degree

#sky_RA_WORLD = SkyCoord('20:44:09 -10:42:54', unit=(u.hourangle, u.deg)).ra.degree #MARK509
#sky_DEC_WORLD = SkyCoord('20:44:09 -10:42:54', unit=(u.hourangle, u.deg)).dec.degree

#sky_RA_WORLD = SkyCoord('05:16:17 -00:09:08', unit=(u.hourangle, u.deg)).ra.degree #AKN120
#sky_DEC_WORLD = SkyCoord('05:16:17 -00:09:08', unit=(u.hourangle, u.deg)).dec.degree

#sky_RA_WORLD = SkyCoord('13:06:26 -40:25:33', unit=(u.hourangle, u.deg)).ra.degree #ESO323
#sky_DEC_WORLD = SkyCoord('13:06:26 -40:25:33', unit=(u.hourangle, u.deg)).dec.degree

#sky_RA_WORLD = SkyCoord('01:54:38 00:43:41', unit=(u.hourangle, u.deg)).ra.degree #SA093_317
#sky_DEC_WORLD = SkyCoord('01:54:38 00:43:41', unit=(u.hourangle, u.deg)).dec.degree

#sky_RA_WORLD = SkyCoord('08:52:32 -00:39:17', unit=(u.hourangle, u.deg)).ra.degree #SA100_241
#sky_DEC_WORLD = SkyCoord('08:52:32 -00:39:17', unit=(u.hourangle, u.deg)).dec.degree


In [13]:
for i in range(len(quasJ)):
    print i, quasJ[i]
    data1, header = fits.getdata(quasJ[i],header=True) #RA---TAN data2 = data1
    data1 = np.array(data1,dtype='float')
    data2, header2 = fits.getdata(quasJ[i],header=True)
    data2 = np.array(data2,dtype='float')
    #print data1
    #print header['CTYPE2']
    #header['CTYPE1'] = "RA---TAN -SIP"
    #header['CTYPE2'] = "DEC--TAN -SIP"
    #print data1
    back = np.ones((header['NAXIS1'],header['NAXIS2']))
    a_sky = curve(data1,header,WCS(quasJ[i]),
                   (sky_RA_WORLD,sky_DEC_WORLD),0.003)[0]
    a_sky_mean = a_sky[(a_sky!=0) & (a_sky<1700)].mean()
    back[data2 == 0] *= 0
    back[data2 != 0] *= a_sky_mean
    #print a_sky_mean
    #data1 = np.subtract(data1,back)
    #print data1
    data2 = data2 - back
    #print data2
    #print data2
    #for j in range(len(np.shape(data[0]))):
    #    for k in range(len(np.shape(data[0]))):
    #        if data[j,k] != 0:
    #            data[j,k] = float(data[j,k]) - a_sky_mean
    #header['NDIT'] = 5
    '''header['A_ORDER'] = "0"
    header['B_ORDER'] = "0"
    header['AP_ORDER'] = "0"
    header['BP_ORDER'] = "0"
    header['A_0_2'] = "0"
    header['A_1_1'] = "0"
    header['A_2_0'] = "0"
    header['B_0_2'] = "0"
    header['B_1_1'] = "0"
    header['B_2_0'] = "0"
    header['AP_0_2'] = "0"
    header['AP_1_1'] = "0"
    header['AP_2_0'] = "0"
    header['AP_0_1'] = "0"
    header['AP_1_0'] = "0"
    header['BP_0_1'] = "0"
    header['BP_0_2'] = "0"
    header['BP_1_0'] = "0"
    header['BP_1_1'] = "0"
    header['BP_2_0'] = "0"'''
    fits.writeto(quasJ[i],data2,header,clobber=True)
    #print fits.getheader(quasH[i])['A_ORDER']

0 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016119BLs025.fits
1 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016119BLs026.fits
2 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016119BLs027.fits
3 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016127BLs030.fits
4 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016127BLs031.fits
5 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016127BLs032.fits
6 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016134BLs063.fits
7 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016134BLs064.fits
8 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016134BLs065.fits
9 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016141BLs052.fits
10 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016141BLs053.fits
11 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016141BLs054.fits
12 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016158BLs046.fits
13 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016158BLs047.fits
14 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016158BLs048.fits
15 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2017046BLs069.fits
16 /media/lynge/Ly

/home/lynge/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: Mean of empty slice.
  from ipykernel import kernelapp as app


119 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016304BLs041.fits
120 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016312BLs106.fits
121 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016312BLs107.fits
122 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016312BLs108.fits
123 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016322BLs114.fits
124 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016322BLs115.fits
125 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016322BLs116.fits
126 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016330BLs066.fits
127 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2016330BLs067.fits
128 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2017210BLs022.fits
129 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2017300BLs113.fits
130 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2017300BLs114.fits
131 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2017300BLs115.fits
132 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2017307BLs120.fits
133 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2017307BLs121.fits
134 /media/lynge/Lynge-Back-up/NGC3783/z/IMG2017307BLs1